# Importaciones

In [1]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
import time

4/rwGsb5th4inWuJQwUNRAEnGVpP2KeQJZIovNBCejTlfqrcWrsASjXhE

In [2]:
def error_measures(Yestimado, Yteorico):
    
    CM = confusion_matrix(Yteorico, Yestimado)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    esp = TN/(TN+FP)
    
    return sens, esp

In [3]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [4]:
df = pd.read_excel('Opiniones_Hoteles_Medellin.xlsx')
df.columns=['texto','sentimiento']

In [5]:
X=df.iloc[:,0]
y=df.iloc[:,1]

In [6]:
vector=CountVectorizer(ngram_range=(1, 2))

In [7]:
vector.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [8]:
bagOfWords=vector.transform(X)

# Regresión logística

In [9]:
from sklearn.linear_model import LogisticRegression
import scipy as sc

In [84]:
def RegresionLogistica(Cvalue):    
    kf = KFold(n_splits=10)
    lr = LogisticRegression(C = Cvalue)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(10)
    tiempo_i = time.time()
    
    j=0
    for train_index, test_index in kf.split(bagOfWords):
        Xtrain, Xtest = bagOfWords[train_index], bagOfWords[test_index]
        Ytrain, Ytest = y[train_index], y[test_index]  

        lr.fit(Xtrain,Ytrain)
        Yest = lr.predict(Xtest)
        s, e = error_measures(Yest, Ytest)
        sens.append(s); esp.append(e)
        acc.append(lr.score(Xtest,Ytest))   
        
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp), np.mean(Errores), time.time()-tiempo_i

In [88]:
import pandas as pd
import qgrid
randn = np.random.randn

tasas = pd.Series(['1.0', '0.9', '0.7', '0.6', '0.5', '0.4', '0.3', '0.2', '0.1', '0.09', '0.08', '0.07', '0.06', '0.05','0.04', '0.03', '0.02', '0.01'])

df_types = pd.DataFrame({
    'Tasa de aprendizaje' : tasas})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Tasa de aprendizaje'], inplace=True)
i=0

for eta in [1.0, 0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05,0.04, 0.03, 0.02, 0.01]:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=RegresionLogistica(eta)       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
    i=i+1
        

# df_types["Error_Entrenamiento"][2] = "0.0"
# df_types["Error_Prueba"][2] = "0.5"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [89]:
qgrid_widget.get_changed_df()

,Accuracy,Int_Accuracy,Sensibility,Int_Sensibility,Especificity,Int_Especificity,Error validación,Tiempo ejecución
Tasa de aprendizaje,,,,,,,,
1.0,0.848605,0.0593352,NaN,NaN,NaN,NaN,0.151395,0.108099
0.9,0.850566,0.0581382,NaN,NaN,NaN,NaN,0.149434,0.0920832
0.7,0.844721,0.0574067,NaN,NaN,NaN,NaN,0.155279,0.0890806
0.6,0.844759,0.059818,NaN,NaN,NaN,NaN,0.155241,0.0860779
0.5,0.844759,0.059818,NaN,NaN,NaN,NaN,0.155241,0.091083
0.4,0.842798,0.0568743,NaN,NaN,NaN,NaN,0.157202,0.0960877
0.3,0.840875,0.060207,NaN,NaN,NaN,NaN,0.159125,0.087079
0.2,0.84276,0.0649014,NaN,NaN,NaN,NaN,0.15724,0.0780711
0.1,0.840799,0.0663257,NaN,NaN,NaN,NaN,0.159201,0.0720651


# KNN

In [61]:
from sklearn.neighbors import KNeighborsClassifier

In [62]:
def vecinos(k):

    KNN_model = KNeighborsClassifier(n_neighbors=k) 

    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0
    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        KNN_model.fit(Xtrain,Ytrain)
        Yest = KNN_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(KNN_model.score(Xtest,Ytest))
        
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [91]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Numero de vecinos' : pd.Series([1, 2, 3, 4, 5, 6, 7, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Numero de vecinos'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=vecinos(i)       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [92]:
qgrid_widget.get_changed_df()

,Accuracy,Int_Accuracy,Sensibility,Int_Sensibility,Especificity,Int_Especificity,Error validación,Tiempo ejecución
Numero de vecinos,,,,,,,,
1,0.679767,0.0436826,0.783296,0.0988136,0.580226,0.120041,0.320233,1.07897
2,0.697752,0.0380792,0.559769,0.125513,0.836971,0.0928052,0.302248,1.097
3,0.699302,0.0463083,0.768644,0.121609,0.6317,0.152797,0.300698,1.14804
4,0.699845,0.0418185,0.636781,0.0938485,0.766628,0.0844821,0.300155,1.21711
5,0.692171,0.0453511,0.788497,0.103814,0.597684,0.125278,0.307829,1.17006
6,0.686202,0.0484292,0.659669,0.13989,0.71608,0.117169,0.313798,1.17706
7,0.688295,0.0525681,0.815924,0.114142,0.56473,0.146021,0.311705,1.16906
10,0.696822,0.0459121,0.716135,0.144771,0.680783,0.136308,0.303178,1.18307
20,0.677054,0.068658,0.908955,0.0890012,0.452257,0.170904,0.322946,1.2001


# Árbol de decisión


In [23]:
from sklearn.tree import DecisionTreeClassifier as DT

In [24]:
def arbol_decision(prof):
    if(prof == 0):
        DT_model = DT(max_depth=None) 
    else:
        DT_model = DT(max_depth = prof)
    
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0
    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        DT_model.fit(Xtrain,Ytrain)
        Yest = DT_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(DT_model.score(Xtest,Ytest))
        
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [28]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Maxima profundidad' : pd.Series([0,5,10,20,30,50,60,70,80,100,200])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Maxima profundidad'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=arbol_decision(i)       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [26]:
qgrid_widget.get_changed_df()

,Accuracy,Int_Accuracy,Sensibility,Int_Sensibility,Especificity,Int_Especificity,Error validación,Tiempo ejecución
Maxima profundidad,,,,,,,,
0,0.801473,0.0359759,0.783527,0.0537222,0.820673,0.0555066,0.198527,1.8667
5,0.798527,0.0350212,0.739353,0.0628442,0.858683,0.0754876,0.201473,1.13102
10,0.799767,0.0348933,0.754525,0.0509587,0.84702,0.070236,0.200233,1.58645
20,0.802403,0.0280251,0.776758,0.0530684,0.82912,0.0535419,0.197597,1.82765
30,0.802093,0.0343097,0.786057,0.0582778,0.81924,0.062677,0.197907,1.88271
50,0.802713,0.030448,0.78039,0.0527565,0.826249,0.0481187,0.197287,1.89372
60,0.800543,0.0395099,0.776159,0.0504729,0.825706,0.0694893,0.199457,1.8747
70,0.806667,0.0321087,0.796299,0.0525613,0.819506,0.0613129,0.193333,1.89873
80,0.803721,0.0329595,0.787713,0.0607321,0.821595,0.054803,0.196279,1.89873


# Random forest


In [29]:
from sklearn.ensemble import RandomForestClassifier as RF

In [30]:
def random_forest(est, carac):
    
    RF_model = RF(n_estimators=est, max_features =carac)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        RF_model.fit(Xtrain,Ytrain)
        Yest = RF_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(RF_model.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1
        
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [31]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,5,5,10,10,10,10,10,10,20,20,20,20,20,20,50,50,50,50,50,50,100,100,100,100,100,100,200,200,200,200,200,200]), 
    'Variables analizadas por nodo' : pd.Series([50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Numero de arboles', 'Variables analizadas por nodo'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=random_forest(i[0], i[1])       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy  \
Numero de arboles Variables analizadas por nodo                          
5                 50                              0.78186    0.0422466   
                  100                            0.797132    0.0375478   
                  150                             0.80186    0.0324346   
                  200                            0.806899    0.0377559   
                  250                            0.811938    0.0358877   
                  300                            0.809612    0.0321368   
10                50                             0.820698    0.0388163   
                  100                            0.834031    0.0329452   
                  150                            0.835194    0.0376048   
                  200                            0.831628    0.0379937   
                  250                            0.836589    0.0304145   
                  300                            0.842016    0.0327932   
20                50                             0.845271    0.0349941   
                  100                            0.853023    0.0282051   
                  150                            0.856744    0.0276795   
                  200                            0.849535    0.0323748   
                  250                            0.852713    0.0296607   
                  300                            0.852093    0.0291638   
50                50                             0.870853    0.0317019   
                  100                            0.865194    0.0354204   
                  150                            0.867907    0.0261027   
                  200                            0.861938    0.0298363   
                  250                            0.860775    0.0329785   
                  300                            0.862713    0.0305252   
100               50                             0.875349    0.0302361   
                  100                            0.870465    0.0302682   
                  150                             0.86814    0.0314598   
                  200                            0.860388      0.03047   
                  250                            0.863101    0.0320225   
                  300                            0.860465    0.0286926   
200               50                             0.875116    0.0286275   
                  100                             0.87031     0.031063   
                  150                            0.870853     0.029523   
                  200                            0.868372    0.0305977   
                  250                            0.867364    0.0310445   
                  300                            0.866977    0.0300575   

                                                Sensibility Int_Sensibility  \
Numero de arboles Variables analizadas por nodo                               
5                 50                               0.837142       0.0649389   
                  100                              0.826036       0.0573304   
                  150                              0.821537       0.0618153   
                  200                              0.825284       0.0715546   
                  250                              0.814093       0.0616129   
                  300                              0.827997       0.0651991   
10                50                               0.814478       0.0604297   
                  100                              0.807452       0.0657875   
                  150                              0.801636        0.068807   
                  200                              0.799966       0.0627805   
                  250                              0.804192        0.061182   
                  300                              0.814137       0.0669705   
20                50                               0.865965       0.0506045   
                  100                              

# Support vector machine

In [33]:
from sklearn import svm

In [34]:
def SVM (ker, c, gam): 
    
    if gam == 0:
        gam = 'auto'
    
    svm_model =svm.SVC(kernel=ker, C=c, gamma=gam)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        svm_model.fit(Xtrain,Ytrain)
        Yest = svm_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(svm_model.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [36]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,200,300,0.001,0.001,0.001,0.001,0.01,0.01,0.01,0.01,0.1,0.1,0.1,0.1,1,1,1,1,10,10,10,10,100,100,100,100,200,200,200,200,300,300,300,300]),
    'gamma' : pd.Series([0,0,0,0,0,0,0,0,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=SVM(i[0], i[1], i[2])       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [37]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy Sensibility Int_Sensibility  \
Kernel C       gamma                                                      
linear 0.001   0.000  0.613178    0.0907308    0.910885        0.203575   
       0.010   0.000  0.801705    0.0343749    0.939009       0.0263683   
       0.100   0.000  0.876899    0.0284461    0.909747       0.0319434   
       1.000   0.000  0.879457    0.0282999    0.886978       0.0454565   
       10.000  0.000  0.881705    0.0252532    0.889757       0.0379881   
       100.000 0.000   0.88031    0.0247169    0.887845       0.0401755   
       200.000 0.000   0.88155    0.0252404    0.887546       0.0346238   
       300.000 0.000  0.881008    0.0268619    0.891851       0.0375008   
rbf    0.001   0.001  0.472093    0.0228781        0.61         0.48775   
               0.010  0.467984    0.0276399        0.65         0.47697   
               0.100  0.471628     0.023119        0.58        0.493559   
               1.000  0.472791    0.0214126        0.57        0.495076   
       0.010   0.001  0.466744    0.0249823        0.56        0.496387   
               0.010      0.47    0.0251587        0.59        0.491833   
               0.100  0.473333    0.0255419        0.67        0.470213   
               1.000  0.473721    0.0219777        0.57        0.495076   
       0.100   0.001  0.465891    0.0247213        0.49          0.4999   
               0.010  0.656667    0.0417065    0.859899       0.0647846   
               0.100  0.473876    0.0261427    0.503106        0.493878   
               1.000   0.46969    0.0247114        0.58        0.493559   
       1.000   0.001  0.690388    0.0524826    0.949483       0.0331235   
               0.010  0.825736    0.0300927    0.898328       0.0383883   
               0.100  0.709225    0.0439542    0.553373       0.0680222   
               1.000  0.479147    0.0278357    0.536346        0.492427   
       10.000  0.001  0.831705     0.030029    0.915876       0.0356414   
               0.010  0.877674    0.0266869    0.870502       0.0389926   
               0.100  0.722171    0.0384284    0.544824       0.0576453   
               1.000   0.47876    0.0225239    0.519134        0.490706   
       100.000 0.001   0.87938     0.026523    0.885712       0.0369079   
               0.010  0.872791    0.0257825    0.868957       0.0421782   
               0.100  0.732636    0.0375548    0.560124       0.0605262   
               1.000  0.483643    0.0211415    0.697193        0.451825   
       200.000 0.001  0.880465    0.0280826    0.887763       0.0444236   
               0.010  0.875581    0.0233429    0.863035       0.0457277   
               0.100  0.729147    0.0431027    0.552754       0.0665779   
               1.000  0.473798    0.0271491    0.673298        0.463073   
       300.000 0.001  0.878915    0.0247766    0.886324       0.0359072   
               0.010  0.878682    0.0270403     0.86618       0.0414027   
               0.100  0.729767    0.0393753    0.556582         0.06118   
               1.000  0.480233    0.0256254    0.588351        0.482196   

                     Especificity Int_Especificity Error validación  \
Kernel C       gamma                                                  
linear 0.001   0.000     0.342276         0.285394         0.386822   
       0.010   0.000     0.667171        0.0563749         0.198295   
       0.100   0.000     0.843444        0.0471681         0.123101   
       1.000   0.000     0.872824        0.0461595         0.120543   
       10.000  0.000     0.874197        0.0446603         0.118295   
       100.000 0.000     0.873168        0.0445101          0.11969   
       200.000 0.000      0.87531        0.0431118          0.11845   
       300.000 0.000     0.870668        0.0409014         0.118992   
rbf    0.001   0.001         0.39          0.48775         0.527907   
               0.010         0.35          0.47697         0.532016   
               0.10

# Red neuronal

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

In [26]:
def red_neuronal(nCO, nPC):
    if(nCO==1):
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC))
    else:
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC,nPC))
        
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        mlp.fit(Xtrain,Ytrain)
        Yest = mlp.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(mlp.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [28]:
randn = np.random.randn

df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=red_neuronal(i[0], i[1])       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy Sensibility  \
N. de capas ocultas Neuronas por capa                                      
1                   20                 0.893178    0.0255127    0.929231   
                    24                 0.894651    0.0261528      0.9263   
                    28                  0.89062    0.0236121    0.924056   
                    32                 0.893256    0.0247056    0.927775   
                    36                 0.893721    0.0212662    0.927769   
2                   20                  0.90062    0.0249603    0.923908   
                    24                 0.903953    0.0273391    0.930186   
                    28                  0.89969    0.0222474     0.92538   
                    32                 0.905581    0.0235225    0.927005   
                    36                 0.899302    0.0252741    0.913357   

                                      Int_Sensibility Especificity  \
N. de capas ocultas Neuronas por capa                                
1                   20                      0.0333343     0.858803   
                    24                      0.0316221     0.863591   
                    28                      0.0321194     0.857206   
                    32                      0.0328846     0.860083   
                    36                      0.0301056      0.86018   
2                   20                      0.0330248     0.878096   
                    24                      0.0327481     0.878231   
                    28                      0.0335615     0.875296   
                    32                      0.0316055     0.884653   
                    36                      0.0377749     0.886256   

                                      Int_Especificity Error validación  \
N. de capas ocultas Neuronas por capa                                     
1                   20                       0.0544717         0.106822   
                    24                       0.0506733         0.105349   
                    28                       0.0415838          0.10938   
                    32                       0.0434086         0.106744   
                    36                       0.0400631         0.106279   
2                   20                       0.0424898        0.0993798   
                    24                       0.0524997        0.0960465   
                    28                       0.0452683          0.10031   
                    32                       0.0422383        0.0944186   
                    36                       0.0432169         0.100698   

                                      Tiempo ejecución  
N. de capas ocultas Neuronas por capa                   
1                   20                         325.637  
                    24                         366.455  
                    28                         400.448  
                    32                         438.435  
                    36                         470.956  
2                   20                         220.724  
                    24                         247.792  
                    28                         263.301  
                    32                         279.846  
                    36                         290.822